In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import math

df= pd.read_csv("economics.csv",index_col="date",parse_dates=True)
# --- Option B : Charger vos données réelles ---
# df = pd.read_csv('unemployment.csv', parse_dates=['Date'])
# Assurez-vous que vos données sont triées par date
# df = df.sort_values('Date')

print("Aperçu des données :")
print(df.head())

# On ne garde que la colonne des valeurs (nombre de chômeurs)
data = df['Unemployment']

# ---------------------------------------------------------
# 2. NORMALISATION ET CRÉATION DE SÉQUENCES
# ---------------------------------------------------------

# Normalisation entre 0 et 1 (Crucial pour LSTM)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Définir la taille de l'entraînement (80% train, 20% test)
training_data_len = int(np.ceil(len(scaled_data) * 0.8))

# Création des données d'entraînement
train_data = scaled_data[0:int(training_data_len), :]
test_data=scaled_data[:]

# Création des données de test
test_data = scaled_data[training_data_len - time_steps: , :]
x_test = []
y_test = data[training_data_len:, :] # Les vraies valeurs non normalisées

for i in range(time_steps, len(test_data)):
    x_test.append(test_data[i-time_steps:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Prédictions
predictions = model.predict(x_test)
# Inversion de la normalisation pour revenir aux vraies unités (nombre de personnes)
predictions = scaler.inverse_transform(predictions)

# Calcul de l'erreur (RMSE)
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
print(f"\nRoot Mean Squared Error (RMSE): {rmse:.2f}")

# ---------------------------------------------------------
# 6. VISUALISATION
# ---------------------------------------------------------

train = df[:training_data_len]
valid = df[training_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(16,8))
plt.title('Modèle LSTM - Prédiction du Chômage aux USA')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Nombre de Chômeurs', fontsize=18)
plt.plot(train['Date'], train['Unemployment'], label='Données Entraînement')
plt.plot(valid['Date'], valid['Unemployment'], label='Valeurs Réelles')
plt.plot(valid['Date'], valid['Predictions'], label='Prédictions LSTM')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

: 

In [ ]:
# Fonction pour créer les séquences (X = passé, y = futur)
def create_sequences(dataset, time_steps=60):
    X = []
    y = []
    for i in range(time_steps, len(dataset)):
        X.append(dataset[i-time_steps:i, 0])
        y.append(dataset[i, 0])
    return np.array(X), np.array(y)

# Fenêtre de temps (Look-back period) : on regarde les 60 derniers mois pour prédire le suivant
time_steps = 60

x_train, y_train = create_sequences(train_data, time_steps)

# Reshape pour LSTM : [samples, time steps, features]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
model = Sequential()

# Couche 1 : LSTM
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2)) # Dropout pour éviter le surapprentissage

# Couche 2 : LSTM
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

# Couche Dense (Sortie)
model.add(Dense(units=25))
model.add(Dense(units=1)) # Prédiction d'une seule valeur

# Compilation
model.compile(optimizer='adam', loss='mean_squared_error')
print("\nDébut de l'entraînement...")
model.fit(x_train, y_train, batch_size=32, epochs=20, verbose=1)